<a href="https://colab.research.google.com/github/bandyg/llamapb/blob/main/llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [ ]:
!pip install llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing inst

In [ ]:
'''create openai instance, set api key and server
   create prompt template
   load the document
   create a functn to ask a question
   create a function to ask a question stream
'''
import os
from dotenv import load_dotenv, find_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

# Load environment variables
load_dotenv(find_dotenv())

# Get API key and base URL from environment variables
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("BASE_URL")
#print("Current working directory:", os.getcwd())
# Initialize OpenAI LLM
llm = OpenAI(model="gpt-3.5-turbo", temperature=0, api_key=api_key, api_base=base_url)

# Define prompt templates
text_qa_template = PromptTemplate(
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

refine_template = PromptTemplate(
    "We have the opportunity to refine the original answer "
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{context_msg}\n"
    "------------\n"
    "Given the new context, refine the original answer to better "
    "answer the question: {query_str}. "
    "If the context isn't useful, output the original answer again.\n"
    "Original Answer: {existing_answer}"
)

# Load the document
with open("./docs/getting_started/starter_example.md", "r") as f:
    text = f.read()

# Function to ask a question
def ask_question(question):
    prompt = text_qa_template.format(context_str=text, query_str=question)
    response = llm.complete(prompt)
    return response.text

def ask_question_stream(question):
    prompt = text_qa_template.format(context_str=text, query_str=question)
    response_gen = llm.stream_complete(prompt)
    for response in response_gen:
        print(response.delta, end="")

# Example usage
if __name__ == "__main__":
    question = "How can I install llama-index repo?"
    print(f"Question: {question}")
    answer = ask_question_stream(question)